In [ ]:
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=618909ec327f7f56aba9c18948c8d8d79d2bce7b37c3980e7baa036e190e42a6
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words


In [ ]:
#Import the required package.
import pandas as pd
import datetime
import json
import re
#import stop_words
import numpy as np

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [ ]:
from stop_words import get_stop_words
from nltk.corpus import stopwords

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Read the file.
df = pd.read_csv("https://raw.githubusercontent.com/MatticusBa/consultinggroup/main/DATA-462%20Loqbox%20Analytics%20Project%20-%20Membership%20Downgrades%20Data%20(1).csv?token=GHSAT0AAAAAAB6DKJ2W5A53SX7P64ZONFEMY6X7Q2Q")

df

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]"
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,"[{""note"": ""`Member wants to proceed with Save ..."
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ..."
3,332,2022-08-05 23:25:15.000,2022-08-25 15:52:59.000,1,1,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""..."
4,445,2022-05-27 07:41:27.000,2022-05-27 07:43:36.000,1,0,0,1,"[{""value"": ""0"", ""reason"": ""My current plan is ..."
...,...,...,...,...,...,...,...,...
45059,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""..."
45060,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]"
45061,784914,2022-12-16 13:49:49.000,2022-12-16 13:50:57.000,0,0,0,0,"[{""value"": ""1"", ""reason"": null}]"
45062,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]"


In [ ]:
#Know data cleaning bugs:
#   - Find a way to merge the reason-NaN column with Notes column

In [ ]:
class data_cleaning:

  def __init__(self, df):
    self.df = df

    #View the dataframe head.
  def view_df(self):
    self.df
    return self.df

    #Remove invalid rows.
  def remove_invalid_rows(self):
    self.df = self.df[(self.df['activated_save'] != 0) |
        (self.df['activated_spend'] != 0) |
        (self.df['activated_rent'] != 0) |
        (self.df['activated_coach'] != 0)]
    return self.df

    #Output the value and reason from the update_reason column as two new columns separately.
  def separate_update_reason(self):
    self.df["update_reason_dict"] = self.df["update_reason"].str[2:].str[:-2]
    self.df["update_reason_dict"] = self.df["update_reason_dict"].str.split("}, {")
    return self.df

    #Split observations with multiple values in update_reason column into multiple rows and concat them back together.
  def explode(self):
    self.df = self.df.explode("update_reason_dict")
    self.df["update_reason_dict"] = "{" + self.df["update_reason_dict"] + "}"
    a = self.df[self.df.update_reason_dict == "{}"]
    b = self.df[self.df.update_reason_dict != "{}"].dropna()
    c = self.df[self.df.update_reason_dict.isna()]
    b = pd.concat([b, b.update_reason_dict.apply(json.loads).apply(pd.Series)], axis=1)
    self.df = pd.concat([a,b,c])[["customer_id", "upgraded_at", "downgraded_at", "activated_save", "activated_spend","activated_rent", "activated_coach", "update_reason", "update_reason_dict", "value", "reason", "note"]].sort_values(by=["customer_id"])
    return self.df

    #Convert the value column to numeric.
  def to_pd_numeric(self):
    self.df.value = pd.to_numeric(self.df.value)
    return self.df

    #Split value and reason into multiple columns from 0-7.
  def separate_value_reason(self):
    c = self.df.copy()
    self.df['value_str']=self.df['value'].astype(str)
    d = self.df.copy()
    self.df = self.df.groupby(["customer_id","upgraded_at"],as_index=False).apply(lambda x:[','.join(x['value_str'])])
    self.df = pd.DataFrame(self.df)
    self.df.columns=['customer_id','upgraded_at','values']
    self.df = pd.merge(c, self.df)
    for i in range(8):
      self.df.loc[self.df['values'].str.contains(str(i)),["value_"+str(i)]]="1"
      self.df["value_"+str(i)] = self.df["value_"+str(i)].fillna("0")
    d = d.pivot(index=['customer_id','upgraded_at','note'], columns='value', values=['reason'])
    d.columns = ["_".join([str(i)for i in col]).strip() for col in d.columns.values]
    d = d.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
    self.df = pd.merge(self.df, d)
    self.df = self.df.drop(columns=['value'])
    return self

    #This function just reorders the columns correctly
  def reorder_columns(self):
    value_cols = [col for col in self.df.columns if 'value_' in col]
    reason_cols = [col for col in self.df.columns if 'reason_' in col]
    dropcols = value_cols + reason_cols
    cols = [col for col in self.df.columns if col not in dropcols]
    new_order = []
    for i in range(len(value_cols)):
        new_order.append('value_'+str(i)+'')
        new_order.append('reason_'+str(i)+'.0')
    new_order.append('reason_nan')
    self.df = self.df.reindex(columns= cols + new_order)
    return self

    #Classificate according to whether the reason is filled in by the users themselves.
  def note_if(self):
    self.df['note t/f'] = self.df["update_reason"].str.startswith(('[{"note"', '[{"reason"'))
    return self

    #Classificate observations by the length of time a user has maintained membership.
  def interval(self):
    self.df['upgrade_date'] = pd.to_datetime(self.df['upgraded_at']).dt.date
    self.df['downgrade_date'] = pd.to_datetime(self.df['downgraded_at']).dt.date
    a = self.df['downgrade_date'] - self.df['upgrade_date']
    self.df['interval']=a.astype('timedelta64[D]').astype(int)
    return self

  def membership_short(self):
    self.df = self.df[self.df['interval']<=14]
    self.df = self.df[~(self.df['interval']>14)]
    return self

    #Classificate dataframces by datetime format.
  def format_old(self):
    self.df['downgrade_date'] = pd.to_datetime(self.df['downgraded_at'])
    dt = datetime.datetime(2022,7,20,14,30)
    self.df = self.df[(self.df['downgrade_date'] < dt)]
    return self

  def format_new(self):
    self.df['downgrade_date'] = pd.to_datetime(self.df['downgraded_at'])
    dt = datetime.datetime(2022,7,20,14,30)
    self.df = self.df[(self.df['downgrade_date'] > dt)]
    return self

  def to_dataframe(self):
    data_dict = self.__dict__
    df = pd.DataFrame.from_dict(data_dict, orient='columns')
    return df



In [ ]:
dcn = data_cleaning(df)
dco = data_cleaning(df)

for obj in [dcn, dco]:
    obj.remove_invalid_rows()
    obj.separate_update_reason()
    obj.explode()
    obj.to_pd_numeric()
    obj.separate_value_reason()
    obj.reorder_columns()
    obj.note_if()
    obj.interval()
    obj.membership_short()

dcn.format_new()
dco.format_old()

<ipython-input-8-3b8e43ae7eb0>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["update_reason_dict"] = self.df["update_reason"].str[2:].str[:-2]
<ipython-input-8-3b8e43ae7eb0>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["update_reason_dict"] = self.df["update_reason_dict"].str.split("}, {")


In [ ]:
dcn.df

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,reason,note,...,reason_5.0,value_6,reason_6.0,value_7,reason_7.0,reason_nan,note t/f,upgrade_date,downgrade_date,interval
9,454,2022-11-04 19:24:14.000,2022-11-04 20:56:40.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",None,NaN,...,None,0,NaN,0,NaN,NaN,False,2022-11-04,2022-11-04 20:56:40,0
18,894,2022-09-10 09:04:00.000,2022-09-10 09:25:23.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...",None,NaN,...,NaN,0,NaN,0,NaN,NaN,False,2022-09-10,2022-09-10 09:25:23,0
40,1814,2022-09-17 14:59:26.000,2022-09-17 15:17:59.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",None,NaN,...,None,0,NaN,0,NaN,NaN,False,2022-09-17,2022-09-17 15:17:59,0
52,2516,2022-11-10 21:38:56.000,2022-11-12 13:18:13.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]",None,NaN,...,NaN,0,NaN,0,NaN,NaN,False,2022-11-10,2022-11-12 13:18:13,2
70,3009,2022-08-10 13:57:31.000,2022-08-17 09:20:52.000,1,0,0,0,"[{""value"": ""2"", ""reason"": null}]",None,NaN,...,NaN,0,NaN,0,NaN,NaN,False,2022-08-10,2022-08-17 09:20:52,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55423,784693,2022-12-16 09:52:04.000,2022-12-16 10:34:04.000,1,0,0,0,"[{""value"": ""1"", ""reason"": ""It's sneaky to offe...",It's sneaky to offer free help then charge for...,NaN,...,It's sneaky to offer free help then charge for...,0,NaN,0,NaN,NaN,False,2022-12-16,2022-12-16 10:34:04,0
55428,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...",None,NaN,...,NaN,0,NaN,0,NaN,NaN,False,2022-12-16,2022-12-16 11:43:52,0
55430,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",None,NaN,...,None,0,NaN,0,NaN,NaN,False,2022-12-16,2022-12-16 12:14:22,0
55431,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]",None,NaN,...,NaN,0,NaN,0,NaN,NaN,False,2022-12-16,2022-12-16 14:23:11,0


In [ ]:
df = df[(df['activated_save'] != 0) |
        (df['activated_spend'] != 0) |
        (df['activated_rent'] != 0) |
        (df['activated_coach'] != 0)]
df

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]"
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,"[{""note"": ""`Member wants to proceed with Save ..."
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ..."
3,332,2022-08-05 23:25:15.000,2022-08-25 15:52:59.000,1,1,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""..."
4,445,2022-05-27 07:41:27.000,2022-05-27 07:43:36.000,1,0,0,1,"[{""value"": ""0"", ""reason"": ""My current plan is ..."
...,...,...,...,...,...,...,...,...
45058,784693,2022-12-16 09:52:04.000,2022-12-16 10:34:04.000,1,0,0,0,"[{""value"": ""1"", ""reason"": ""It's sneaky to offe..."
45059,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""..."
45060,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]"
45062,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]"


In [ ]:
#Split the update_reason column with multiple values and reasons.
df1 = df.copy()

df1["update_reason_dict"] = df1["update_reason"].str[2:].str[:-2]
df1["update_reason_dict"] = df1["update_reason_dict"].str.split("}, {")

df1

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,update_reason_dict
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]","[""value"": ""1"", ""reason"": null]"
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,"[{""note"": ""`Member wants to proceed with Save ...","[""note"": ""`Member wants to proceed with Save o..."
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","[""value"": ""0"", ""reason"": ""My current plan is t..."
3,332,2022-08-05 23:25:15.000,2022-08-25 15:52:59.000,1,1,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","[""value"": ""1"", ""reason"": null, ""value"": ""2"", ""..."
4,445,2022-05-27 07:41:27.000,2022-05-27 07:43:36.000,1,0,0,1,"[{""value"": ""0"", ""reason"": ""My current plan is ...","[""value"": ""0"", ""reason"": ""My current plan is t..."
...,...,...,...,...,...,...,...,...,...
45058,784693,2022-12-16 09:52:04.000,2022-12-16 10:34:04.000,1,0,0,0,"[{""value"": ""1"", ""reason"": ""It's sneaky to offe...","[""value"": ""1"", ""reason"": ""It's sneaky to offer..."
45059,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","[""value"": ""1"", ""reason"": null, ""value"": ""2"", ""..."
45060,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]","[""value"": ""5"", ""reason"": null]"
45062,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]","[""value"": ""2"", ""reason"": null]"


In [ ]:
#Split observations with multiple values in update_reason column into multiple rows.
df2 = df1.explode("update_reason_dict")
df2["update_reason_dict"] = "{" + df2["update_reason_dict"] + "}"

df2

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,update_reason_dict
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]","{""value"": ""1"", ""reason"": null}"
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,"[{""note"": ""`Member wants to proceed with Save ...","{""note"": ""`Member wants to proceed with Save o..."
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","{""value"": ""0"", ""reason"": ""My current plan is t..."
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","{""value"": ""1"", ""reason"": ""My current plan isn’..."
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","{""value"": ""2"", ""reason"": ""I’m not taking advan..."
...,...,...,...,...,...,...,...,...,...
45059,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","{""value"": ""1"", ""reason"": null}"
45059,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","{""value"": ""2"", ""reason"": null}"
45060,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]","{""value"": ""5"", ""reason"": null}"
45062,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]","{""value"": ""2"", ""reason"": null}"


In [ ]:
#Remove the NAN update_reason observations.
df2a = df2[df2.update_reason_dict == "{}"]
df2b = df2[df2.update_reason_dict != "{}"].dropna()
df2c = df2[df2.update_reason_dict.isna()]

In [ ]:
#Output the value and reason from the update_reason column as two new columns separately.
df2b = pd.concat([df2b, df2b.update_reason_dict.apply(json.loads).apply(pd.Series)], axis=1)

df2b

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,update_reason_dict,value,reason,note
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]","{""value"": ""1"", ""reason"": null}",1,None,NaN
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,"[{""note"": ""`Member wants to proceed with Save ...","{""note"": ""`Member wants to proceed with Save o...",2,NaN,`Member wants to proceed with Save only
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","{""value"": ""0"", ""reason"": ""My current plan is t...",0,My current plan is too expensive,NaN
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","{""value"": ""1"", ""reason"": ""My current plan isn’...",1,My current plan isn’t good value for money,NaN
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","{""value"": ""2"", ""reason"": ""I’m not taking advan...",2,I’m not taking advantage of all the benefits o...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
45059,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","{""value"": ""1"", ""reason"": null}",1,None,NaN
45059,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","{""value"": ""2"", ""reason"": null}",2,None,NaN
45060,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]","{""value"": ""5"", ""reason"": null}",5,None,NaN
45062,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]","{""value"": ""2"", ""reason"": null}",2,None,NaN


In [ ]:
#Add NAN update_reason obversations back.
df3 = pd.concat([df2a,df2b,df2c])[["customer_id", "upgraded_at", "downgraded_at", "activated_save", "activated_spend","activated_rent", "activated_coach", "value", "reason", "note"]].sort_values(by=["customer_id"])
df3.value = pd.to_numeric(df3.value)
df3['upgrade_date'] = pd.to_datetime(df3['upgraded_at'])
df3['downgrade_date'] = pd.to_datetime(df3['downgraded_at'])
df3

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,value,reason,note,upgrade_date,downgrade_date
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,1.0,None,NaN,2022-06-06 11:05:27,2022-11-16 20:34:41
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,2.0,NaN,`Member wants to proceed with Save only,2022-02-12 09:23:22,2022-03-14 16:17:01
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,2.0,I’m not taking advantage of all the benefits o...,NaN,2022-03-01 08:49:00,2022-05-08 12:37:56
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,1.0,My current plan isn’t good value for money,NaN,2022-03-01 08:49:00,2022-05-08 12:37:56
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,0.0,My current plan is too expensive,NaN,2022-03-01 08:49:00,2022-05-08 12:37:56
...,...,...,...,...,...,...,...,...,...,...,...,...
45059,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,1.0,None,NaN,2022-12-16 11:28:09,2022-12-16 11:43:52
45059,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,2.0,None,NaN,2022-12-16 11:28:09,2022-12-16 11:43:52
45060,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,5.0,None,NaN,2022-12-16 12:11:55,2022-12-16 12:14:22
45062,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,2.0,None,NaN,2022-12-16 14:14:46,2022-12-16 14:23:11


In [ ]:
#Add a new column to show which values the user selected
df3['value_str']=df3['value'].astype(str)
df4=df3.groupby(["customer_id","upgraded_at"],as_index=False).apply(lambda x:[','.join(x['value_str'])])
df4=pd.DataFrame(df4)
df4.columns=['customer_id','upgraded_at','values']
df4=pd.merge(df,df4)
df4

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,values
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]",1.0
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,"[{""note"": ""`Member wants to proceed with Save ...",2.0
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","2.0,1.0,0.0"
3,332,2022-08-05 23:25:15.000,2022-08-25 15:52:59.000,1,1,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","4.0,2.0,1.0"
4,445,2022-05-27 07:41:27.000,2022-05-27 07:43:36.000,1,0,0,1,"[{""value"": ""0"", ""reason"": ""My current plan is ...",0.0
...,...,...,...,...,...,...,...,...,...
44688,784693,2022-12-16 09:52:04.000,2022-12-16 10:34:04.000,1,0,0,0,"[{""value"": ""1"", ""reason"": ""It's sneaky to offe...","1.0,2.0,3.0,4.0,5.0"
44689,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","1.0,2.0"
44690,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",5.0
44691,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]",2.0


In [ ]:
# Xenia's draft
def format_array(df4):
  for i in range(8):
    df4.loc[df4['values'].str.contains(str(i)),["value_"+str(i)]]="1"
    df4["value_"+str(i)]=df4["value_"+str(i)].fillna("0")
  return df4
df4 = format_array(df4)
df4


,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,values,value_0,value_1,value_2,value_3,value_4,value_5,value_6,value_7
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]",1.0,1,1,0,0,0,0,0,0
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,"[{""note"": ""`Member wants to proceed with Save ...",2.0,1,0,1,0,0,0,0,0
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","2.0,1.0,0.0",1,1,1,0,0,0,0,0
3,332,2022-08-05 23:25:15.000,2022-08-25 15:52:59.000,1,1,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","4.0,2.0,1.0",1,1,1,0,1,0,0,0
4,445,2022-05-27 07:41:27.000,2022-05-27 07:43:36.000,1,0,0,1,"[{""value"": ""0"", ""reason"": ""My current plan is ...",0.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44688,784693,2022-12-16 09:52:04.000,2022-12-16 10:34:04.000,1,0,0,0,"[{""value"": ""1"", ""reason"": ""It's sneaky to offe...","1.0,2.0,3.0,4.0,5.0",1,1,1,1,1,1,0,0
44689,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","1.0,2.0",1,1,1,0,0,0,0,0
44690,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",5.0,1,0,0,0,0,1,0,0
44691,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]",2.0,1,0,1,0,0,0,0,0


In [ ]:
#Separate the different reasons into different columns and combine them
df5=df3.copy()
df5=df3.pivot(index=['customer_id','upgraded_at','note'], columns='value', values=['reason'])
df5.columns=["_".join([str(i)for i in col]).strip() for col in df5.columns.values]
df5=df5.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
df5=pd.merge(df4,df5)
df5

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,values,value_0,...,note,reason_nan,reason_0.0,reason_1.0,reason_2.0,reason_3.0,reason_4.0,reason_5.0,reason_6.0,reason_7.0
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]",1.0,1,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,"[{""note"": ""`Member wants to proceed with Save ...",2.0,1,...,`Member wants to proceed with Save only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,267,2022-03-01 08:49:00.000,2022-05-08 12:37:56.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...","2.0,1.0,0.0",1,...,NaN,NaN,My current plan is too expensive,My current plan isn’t good value for money,I’m not taking advantage of all the benefits o...,NaN,NaN,NaN,NaN,NaN
3,332,2022-08-05 23:25:15.000,2022-08-25 15:52:59.000,1,1,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","4.0,2.0,1.0",1,...,NaN,NaN,NaN,None,None,NaN,None,NaN,NaN,NaN
4,445,2022-05-27 07:41:27.000,2022-05-27 07:43:36.000,1,0,0,1,"[{""value"": ""0"", ""reason"": ""My current plan is ...",0.0,1,...,NaN,NaN,My current plan is too expensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44688,784693,2022-12-16 09:52:04.000,2022-12-16 10:34:04.000,1,0,0,0,"[{""value"": ""1"", ""reason"": ""It's sneaky to offe...","1.0,2.0,3.0,4.0,5.0",1,...,NaN,NaN,NaN,It's sneaky to offer free help then charge for...,It's sneaky to offer free help then charge for...,It's sneaky to offer free help then charge for...,It's sneaky to offer free help then charge for...,It's sneaky to offer free help then charge for...,NaN,NaN
44689,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","1.0,2.0",1,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
44690,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",5.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
44691,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]",2.0,1,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Matt's draft - this just reorders the columns correctly
value_cols = [col for col in df5.columns if 'value_' in col]
reason_cols = [col for col in df5.columns if 'reason_' in col]
dropcols = value_cols + reason_cols

cols = [col for col in df5.columns if col not in dropcols]

new_order = []
for i in range(len(value_cols)):
    new_order.append('value_'+str(i)+'')
    new_order.append('reason_'+str(i)+'.0')
new_order.append('reason_nan')

df5_reordered = df5.reindex(columns= cols + new_order)
df5_reordered.columns.tolist()


['customer_id',
 'upgraded_at',
 'downgraded_at',
 'activated_save',
 'activated_spend',
 'activated_rent',
 'activated_coach',
 'update_reason',
 'values',
 'note',
 'value_0',
 'reason_0.0',
 'value_1',
 'reason_1.0',
 'value_2',
 'reason_2.0',
 'value_3',
 'reason_3.0',
 'value_4',
 'reason_4.0',
 'value_5',
 'reason_5.0',
 'value_6',
 'reason_6.0',
 'value_7',
 'reason_7.0',
 'reason_nan']

In [ ]:
df5=df5_reordered


In [ ]:
#Classificate obversations according to old and new feedback systems.
df5['upgrade_date'] = pd.to_datetime(df5['upgraded_at']).dt.date
df5['upgrade_time'] = pd.to_datetime(df5['upgraded_at']).dt.time
df5['downgrade_date'] = pd.to_datetime(df5['downgraded_at'])
df5['downgrade_time'] = pd.to_datetime(df5['downgraded_at']).dt.time

In [ ]:
#Splitting dataframces by datetime format
dt = datetime.datetime(2022,7,20,14,30)
res1 = df5[(df5['downgrade_date'] < dt)]
res2 = df5[(df5['downgrade_date'] >= dt)]
res2

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,values,note,...,reason_5.0,value_6,reason_6.0,value_7,reason_7.0,reason_nan,upgrade_date,upgrade_time,downgrade_date,downgrade_time
0,18,2022-06-06 11:05:27.000,2022-11-16 20:34:41.000,1,0,1,0,"[{""value"": ""1"", ""reason"": null}]",1.0,NaN,...,NaN,0,NaN,0,NaN,NaN,2022-06-06,11:05:27,2022-11-16 20:34:41,20:34:41
3,332,2022-08-05 23:25:15.000,2022-08-25 15:52:59.000,1,1,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","4.0,2.0,1.0",NaN,...,NaN,0,NaN,0,NaN,NaN,2022-08-05,23:25:15,2022-08-25 15:52:59,15:52:59
5,454,2022-11-04 19:24:14.000,2022-11-04 20:56:40.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",5.0,NaN,...,None,0,NaN,0,NaN,NaN,2022-11-04,19:24:14,2022-11-04 20:56:40,20:56:40
7,548,2021-11-16 12:04:23.000,2022-09-20 16:31:40.000,1,0,0,0,"[{""note"": ""member request"", ""value"": ""2""}]",2.0,member request,...,NaN,0,NaN,0,NaN,NaN,2021-11-16,12:04:23,2022-09-20 16:31:40,16:31:40
8,641,2022-03-29 20:16:44.000,2022-12-13 03:11:26.000,1,1,0,0,"[{""value"": ""3"", ""reason"": null}, {""value"": ""4""...","5.0,4.0,3.0",NaN,...,None,0,NaN,0,NaN,NaN,2022-03-29,20:16:44,2022-12-13 03:11:26,03:11:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44688,784693,2022-12-16 09:52:04.000,2022-12-16 10:34:04.000,1,0,0,0,"[{""value"": ""1"", ""reason"": ""It's sneaky to offe...","1.0,2.0,3.0,4.0,5.0",NaN,...,It's sneaky to offer free help then charge for...,0,NaN,0,NaN,NaN,2022-12-16,09:52:04,2022-12-16 10:34:04,10:34:04
44689,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","1.0,2.0",NaN,...,NaN,0,NaN,0,NaN,NaN,2022-12-16,11:28:09,2022-12-16 11:43:52,11:43:52
44690,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",5.0,NaN,...,None,0,NaN,0,NaN,NaN,2022-12-16,12:11:55,2022-12-16 12:14:22,12:14:22
44691,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]",2.0,NaN,...,NaN,0,NaN,0,NaN,NaN,2022-12-16,14:14:46,2022-12-16 14:23:11,14:23:11


In [ ]:
#Classificate observations according to the length of time a user has maintained membership.
df5['upgrade_date'] = pd.to_datetime(df5['upgraded_at']).dt.date
df5['downgrade_date'] = pd.to_datetime(df5['downgraded_at']).dt.date
df5['days'] = df5['downgrade_date'] - df5['upgrade_date']
df5['interval']=df5['days'].astype('timedelta64[D]').astype(int)

gr1=df5[df5['interval']<=14]
gr2=df5[~(df5['interval']<=14)]

gr1

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,values,note,...,reason_6.0,value_7,reason_7.0,reason_nan,upgrade_date,upgrade_time,downgrade_date,downgrade_time,days,interval
4,445,2022-05-27 07:41:27.000,2022-05-27 07:43:36.000,1,0,0,1,"[{""value"": ""0"", ""reason"": ""My current plan is ...",0.0,NaN,...,NaN,0,NaN,NaN,2022-05-27,07:41:27,2022-05-27,07:43:36,0 days,0
5,454,2022-11-04 19:24:14.000,2022-11-04 20:56:40.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",5.0,NaN,...,NaN,0,NaN,NaN,2022-11-04,19:24:14,2022-11-04,20:56:40,0 days,0
9,661,2022-04-03 19:48:13.000,2022-04-07 08:08:38.000,1,0,0,1,NaN,nan,NaN,...,NaN,0,NaN,NaN,2022-04-03,19:48:13,2022-04-07,08:08:38,4 days,4
12,894,2022-09-10 09:04:00.000,2022-09-10 09:25:23.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","4.0,1.0,2.0",NaN,...,NaN,0,NaN,NaN,2022-09-10,09:04:00,2022-09-10,09:25:23,0 days,0
15,1044,2022-02-08 21:11:39.000,2022-02-08 21:13:36.000,1,0,0,0,"[{""value"": ""0"", ""reason"": ""My current plan is ...",0.0,NaN,...,NaN,0,NaN,NaN,2022-02-08,21:11:39,2022-02-08,21:13:36,0 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44688,784693,2022-12-16 09:52:04.000,2022-12-16 10:34:04.000,1,0,0,0,"[{""value"": ""1"", ""reason"": ""It's sneaky to offe...","1.0,2.0,3.0,4.0,5.0",NaN,...,NaN,0,NaN,NaN,2022-12-16,09:52:04,2022-12-16,10:34:04,0 days,0
44689,784773,2022-12-16 11:28:09.000,2022-12-16 11:43:52.000,1,0,0,0,"[{""value"": ""1"", ""reason"": null}, {""value"": ""2""...","1.0,2.0",NaN,...,NaN,0,NaN,NaN,2022-12-16,11:28:09,2022-12-16,11:43:52,0 days,0
44690,784819,2022-12-16 12:11:55.000,2022-12-16 12:14:22.000,1,0,0,0,"[{""value"": ""5"", ""reason"": null}]",5.0,NaN,...,NaN,0,NaN,NaN,2022-12-16,12:11:55,2022-12-16,12:14:22,0 days,0
44691,784930,2022-12-16 14:14:46.000,2022-12-16 14:23:11.000,1,1,0,1,"[{""value"": ""2"", ""reason"": null}]",2.0,NaN,...,NaN,0,NaN,NaN,2022-12-16,14:14:46,2022-12-16,14:23:11,0 days,0


In [ ]:
#Classificate according to whether the reason is filled in by the users themselves. #merge this with the rest of the dataframes
df5['note t/f'] = df5["update_reason"].str.startswith(('[{"note"', '[{"reason"'))

notes = df5[(df5['note t/f']==True)]
vals = df5[(df5['note t/f']==False)]

notes

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,update_reason,values,note,...,value_7,reason_7.0,reason_nan,upgrade_date,upgrade_time,downgrade_date,downgrade_time,days,interval,note t/f
1,252,2022-02-12 09:23:22.000,2022-03-14 16:17:01.000,1,0,0,0,"[{""note"": ""`Member wants to proceed with Save ...",2.0,`Member wants to proceed with Save only,...,0,NaN,NaN,2022-02-12,09:23:22,2022-03-14,16:17:01,30 days,30,True
7,548,2021-11-16 12:04:23.000,2022-09-20 16:31:40.000,1,0,0,0,"[{""note"": ""member request"", ""value"": ""2""}]",2.0,member request,...,0,NaN,NaN,2021-11-16,12:04:23,2022-09-20,16:31:40,308 days,308,True
13,905,2022-09-06 21:11:54.000,2022-10-03 12:48:08.000,1,1,0,0,"[{""note"": ""customer request"", ""value"": ""0""}]",0.0,customer request,...,0,NaN,NaN,2022-09-06,21:11:54,2022-10-03,12:48:08,27 days,27,True
21,1497,2021-11-02 16:19:56.000,2022-07-15 14:34:15.000,1,0,0,0,"[{""note"": ""requested"", ""value"": ""2""}]",2.0,requested,...,0,NaN,NaN,2021-11-02,16:19:56,2022-07-15,14:34:15,255 days,255,True
25,1689,2022-03-13 03:00:48.000,2022-03-14 13:37:00.000,1,1,0,0,"[{""note"": ""Wants Lite"", ""value"": ""3"", ""reason""...",3.0,Wants Lite,...,0,NaN,NaN,2022-03-13,03:00:48,2022-03-14,13:37:00,1 days,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44633,782036,2022-12-13 14:30:33.000,2022-12-14 11:20:31.000,1,1,0,1,"[{""note"": ""member request"", ""value"": ""3"", ""rea...",3.0,member request,...,0,NaN,NaN,2022-12-13,14:30:33,2022-12-14,11:20:31,1 days,1,True
44635,782207,2022-12-13 17:06:28.000,2022-12-13 17:13:57.000,1,0,0,0,"[{""note"": ""Only has lb save"", ""value"": ""2""}]",2.0,Only has lb save,...,0,NaN,NaN,2022-12-13,17:06:28,2022-12-13,17:13:57,0 days,0,True
44677,783874,2022-12-15 11:36:06.000,2022-12-15 12:09:33.000,1,1,0,1,"[{""note"": ""Member request"", ""value"": ""3"", ""rea...",3.0,Member request,...,0,NaN,NaN,2022-12-15,11:36:06,2022-12-15,12:09:33,0 days,0,True
44679,783946,2022-12-15 12:42:48.000,2022-12-15 13:16:21.000,1,0,0,0,"[{""note"": ""Did not mean to sign up"", ""value"": ...",3.0,Did not mean to sign up,...,0,NaN,NaN,2022-12-15,12:42:48,2022-12-15,13:16:21,0 days,0,True


In [ ]:
df5=df5[['customer_id', 'values', 'downgrade_date','downgrade_time','interval','activated_save','activated_spend','activated_rent','activated_coach','value_0','reason_0','value_1','reason_1','value_2','reason_2','value_3','reason_3','value_4','reason_4','value_5','reason_5','value_6','reason_6','value_7','reason_7','note t/f','note']]

df5

,customer_id,downgrade_date,downgrade_time,interval,activated_save,activated_spend,activated_rent,activated_coach,value_0,reason_0,...,value_4,reason_4,value_5,reason_5,value_6,reason_6,value_7,reason_7,note t/f,note
0,18,2022-11-16,20:34:41,163,1,0,1,0,1,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,False,NaN
1,252,2022-03-14,16:17:01,30,1,0,0,0,1,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,True,`Member wants to proceed with Save only
2,267,2022-05-08,12:37:56,68,1,0,0,0,1,My current plan is too expensive,...,0,NaN,0,NaN,0,NaN,0,NaN,False,NaN
3,332,2022-08-25,15:52:59,20,1,1,0,0,1,NaN,...,1,None,0,NaN,0,NaN,0,NaN,False,NaN
4,445,2022-05-27,07:43:36,0,1,0,0,1,1,My current plan is too expensive,...,0,NaN,0,NaN,0,NaN,0,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44688,784693,2022-12-16,10:34:04,0,1,0,0,0,1,NaN,...,1,It's sneaky to offer free help then charge for...,1,It's sneaky to offer free help then charge for...,0,NaN,0,NaN,False,NaN
44689,784773,2022-12-16,11:43:52,0,1,0,0,0,1,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,False,NaN
44690,784819,2022-12-16,12:14:22,0,1,0,0,0,1,NaN,...,0,NaN,1,None,0,NaN,0,NaN,False,NaN
44691,784930,2022-12-16,14:23:11,0,1,1,0,1,1,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,False,NaN


In [ ]:
# Data Preprocessing - Danie's draft

In [ ]:
# Split into 2 sub datasets
system_change = datetime.datetime(2022,7,20,14,30)
df3_old = df3[df3.downgrade_date < system_change]
df3_new = df3[df3.downgrade_date >= system_change]

In [ ]:
# Get stop word list
stop_words = list(get_stop_words('en'))
nltk_words = list(stopwords.words('english'))
stop_words.extend(nltk_words)

In [ ]:
#Remove rows with NaN reason or non-NaN notes
df3_new_1 = df3_new[(df3_new["reason"].notna()) & (df3_new["note"].isnull())]

In [ ]:
#Remove null Reason/dup Reason, lowercase + tokenize + remove stop words + remove punctuation + lemmatize/stem
df3_new_2 = df3_new[(df3_new["reason"].notna()) & (df3_new["note"].isnull())].drop(columns="value").drop_duplicates(keep="first")
df3_new_2["reason_v2"] = df3_new_2["reason"].str.lower().apply(word_tokenize)
df3_new_2["reason_v3"] = df3_new_2["reason_v2"].apply(lambda x: [word for word in x if word.isalpha() and word not in (stop_words)])

#Lemmatize
lmtzr = WordNetLemmatizer()
df3_new_2["reason_lmt"] = df3_new_2["reason_v3"].apply(lambda x: [lmtzr.lemmatize(word) for word in x])

#Stem
ps = PorterStemmer()
df3_new_2["reason_stem"] = df3_new_2["reason_v3"].apply(lambda x: [ps.stem(word) for word in x])
df3_new_2

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,reason,note,upgrade_date,downgrade_date,reason_v2,reason_v3,reason_lmt,reason_stem
32,1828,2022-03-26 11:49:32.000,2022-08-11 07:16:05.000,1,0,0,0,I cancelled this months ago and still got char...,NaN,2022-03-26 11:49:32,2022-08-11 07:16:05,"[i, cancelled, this, months, ago, and, still, ...","[cancelled, months, ago, still, got, charged, ...","[cancelled, month, ago, still, got, charged, l...","[cancel, month, ago, still, got, charg, like, ..."
73,3280,2022-04-15 17:28:38.000,2022-09-16 14:51:58.000,1,0,0,0,It weren’t going up,NaN,2022-04-15 17:28:38,2022-09-16 14:51:58,"[it, weren, ’, t, going, up]",[going],[going],[go]
125,4625,2022-07-31 14:52:07.000,2022-09-03 07:44:51.000,1,0,0,0,I honestly can't afford the extra £2.50 a week...,NaN,2022-07-31 14:52:07,2022-09-03 07:44:51,"[i, honestly, ca, n't, afford, the, extra, £2....","[honestly, ca, afford, extra, week, even, noti...","[honestly, ca, afford, extra, week, even, noti...","[honestli, ca, afford, extra, week, even, noti..."
128,4785,2022-09-22 11:14:27.000,2022-10-06 07:48:26.000,1,0,0,0,It doesn't make sense for me to want to save f...,NaN,2022-09-22 11:14:27,2022-10-06 07:48:26,"[it, does, n't, make, sense, for, me, to, want...","[make, sense, want, save, hardship, yet, spend...","[make, sense, want, save, hardship, yet, spend...","[make, sens, want, save, hardship, yet, spend,..."
167,6191,2022-08-03 09:19:12.000,2022-09-26 19:33:30.000,1,0,0,0,Shall i get my 2.50 a week money back,NaN,2022-08-03 09:19:12,2022-09-26 19:33:30,"[shall, i, get, my, 2.50, a, week, money, back]","[shall, get, week, money, back]","[shall, get, week, money, back]","[shall, get, week, money, back]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45024,782869,2022-12-14 14:09:24.000,2022-12-14 14:26:20.000,0,1,0,0,I want to cancel my full membership and everyt...,NaN,2022-12-14 14:09:24,2022-12-14 14:26:20,"[i, want, to, cancel, my, full, membership, an...","[want, cancel, full, membership, everything, e...","[want, cancel, full, membership, everything, e...","[want, cancel, full, membership, everyth, els,..."
45029,783151,2022-12-14 17:16:23.000,2022-12-15 14:40:46.000,1,0,0,0,Cancel everything even my savings request,NaN,2022-12-14 17:16:23,2022-12-15 14:40:46,"[cancel, everything, even, my, savings, request]","[cancel, everything, even, savings, request]","[cancel, everything, even, saving, request]","[cancel, everyth, even, save, request]"
45031,783375,2022-12-14 23:24:38.000,2022-12-16 00:18:40.000,1,1,1,1,I want to cancel my membership,NaN,2022-12-14 23:24:38,2022-12-16 00:18:40,"[i, want, to, cancel, my, membership]","[want, cancel, membership]","[want, cancel, membership]","[want, cancel, membership]"
45055,784517,2022-12-16 00:33:21.000,2022-12-16 09:17:28.000,1,1,0,1,"£2.50 a week is a lot for this, tbh",NaN,2022-12-16 00:33:21,2022-12-16 09:17:28,"[£2.50, a, week, is, a, lot, for, this, ,, tbh]","[week, lot, tbh]","[week, lot, tbh]","[week, lot, tbh]"


In [ ]:
#Remove rows with NaN reason or non-NaN notes
df3_old_1 = df3_old[(df3_old["reason"].notna()) & (df3_old["note"].isnull())]

#Find written reasons by customers (not suggested by LOQBOX)
distinct_reason = df3_old_1.groupby(['value', 'reason']).size().reset_index(name='counts')
written_reason = distinct_reason[distinct_reason.counts < 20]

In [ ]:
#Remove null Reason/dup Reason, lowercase + tokenize + remove stop words + remove punctuation + lemmatize/stem
df3_old_2 = df3_old[(df3_old["reason"].notna()) & (df3_old["note"].isnull())].drop(columns="value").drop_duplicates(keep="first")
df3_old_2 = df3_old_2[df3_old_2.reason.isin(written_reason.reason)]
df3_old_2["reason_v2"] = df3_old_2["reason"].str.lower().apply(word_tokenize)
df3_old_2["reason_v3"] = df3_old_2["reason_v2"].apply(lambda x: [word for word in x if word.isalpha() and word not in (stop_words)])

lmtzr = WordNetLemmatizer()
df3_old_2["reason_lmt"] = df3_old_2["reason_v3"].apply(lambda x: [lmtzr.lemmatize(word) for word in x])

ps = PorterStemmer()
df3_old_2["reason_stem"] = df3_old_2["reason_v3"].apply(lambda x: [ps.stem(word) for word in x])

In [ ]:
#Combine 2 sub-datasets
df3_preprocessed = pd.concat([df3_new_2, df3_old_2]).drop(columns = ["reason_v2", "reason_v3"])
df3_preprocessed.head(5)

,customer_id,upgraded_at,downgraded_at,activated_save,activated_spend,activated_rent,activated_coach,reason,note,upgrade_date,downgrade_date,reason_lmt,reason_stem
32,1828,2022-03-26 11:49:32.000,2022-08-11 07:16:05.000,1,0,0,0,I cancelled this months ago and still got char...,NaN,2022-03-26 11:49:32,2022-08-11 07:16:05,"[cancelled, month, ago, still, got, charged, l...","[cancel, month, ago, still, got, charg, like, ..."
73,3280,2022-04-15 17:28:38.000,2022-09-16 14:51:58.000,1,0,0,0,It weren’t going up,NaN,2022-04-15 17:28:38,2022-09-16 14:51:58,[going],[go]
125,4625,2022-07-31 14:52:07.000,2022-09-03 07:44:51.000,1,0,0,0,I honestly can't afford the extra £2.50 a week...,NaN,2022-07-31 14:52:07,2022-09-03 07:44:51,"[honestly, ca, afford, extra, week, even, noti...","[honestli, ca, afford, extra, week, even, noti..."
128,4785,2022-09-22 11:14:27.000,2022-10-06 07:48:26.000,1,0,0,0,It doesn't make sense for me to want to save f...,NaN,2022-09-22 11:14:27,2022-10-06 07:48:26,"[make, sense, want, save, hardship, yet, spend...","[make, sens, want, save, hardship, yet, spend,..."
167,6191,2022-08-03 09:19:12.000,2022-09-26 19:33:30.000,1,0,0,0,Shall i get my 2.50 a week money back,NaN,2022-08-03 09:19:12,2022-09-26 19:33:30,"[shall, get, week, money, back]","[shall, get, week, money, back]"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df3_preprocessed.to_csv("df3_preprocessed.csv", index=False)
!cp df3_preprocessed.csv "drive/My Drive/"

In [ ]:
mining_0 = df3_preprocessed.copy()

In [ ]:
# Generate count vectors
word_vec = mining_0['reason_lmt'].apply(pd.value_counts).fillna(0)
word_vec

,aasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasds,abit,able,abo,abroad,absolute,absolutely,abused,acc,accept,...,yeh,yes,yet,yourq,yous,youve,yr,yuysjsjssjjs,yyy,ı
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Compute term frequencies
tf = word_vec.divide(np.sum(word_vec, axis=1), axis=0)
tf

,aasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasds,abit,able,abo,abroad,absolute,absolutely,abused,acc,accept,...,yeh,yes,yet,yourq,yous,youve,yr,yuysjsjssjjs,yyy,ı
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Compute inverse document frequencies
idf = np.log10(len(tf) / word_vec[word_vec > 0].count())
idf

In [ ]:
# Compute TF-IDF vectors
tfidf = np.multiply(tf, idf.to_frame().T)
tfidf

<ipython-input-50-dec3a72396ac>:2: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  tfidf = np.multiply(tf, idf.to_frame().T)


,aasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasdsaasdadsadasdasdasdasdasdasds,abit,able,abo,abroad,absolute,absolutely,abused,acc,accept,...,yeh,yes,yet,yourq,yous,youve,yr,yuysjsjssjjs,yyy,ı
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.137801,0.0,0.0,0.0,0.0,0.0,0.0,0.0
167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

X,y = fetch_20newsgroups(return_X_y = True)
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X).toarray()
vocab = tfidf.vocabulary_
reverse_vocab = {v:k for k,v in vocab.items()}

feature_names = tfidf.get_feature_names()
df_tfidf = pd.DataFrame(X_tfidf, columns = feature_names)

idx = X_tfidf.argsort(axis=1)

tfidf_max10 = idx[:,-10:]

df_tfidf['top10'] = [[reverse_vocab.get(item) for item in row] for row in tfidf_max10 ]

df_tfidf['top10']

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
